In [ ]:
import yahoo_fin.stock_info as si
import pandas as pd
import concurrent.futures
import time
startTime = time.time()
balance_sheets = {}
stock_list = [] #['AMZN']
stock_list.extend(si.tickers_sp500())
print(stock_list)

In [ ]:
income_statements = {}
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ticker in stock_list:
        balance_sheets[ticker] = executor.submit(si.get_balance_sheet,ticker)
#         print(ticker)
print('done')

for ticker in stock_list:
    balance_sheets[ticker] = balance_sheets[ticker].result(60) # 60 sec timeout
#     print(ticker + '2')
    
print('done2')

In [ ]:
recent_sheets = {ticker : sheet.iloc[:,:1] for ticker,sheet in balance_sheets.items()}

for ticker in recent_sheets.keys():
    recent_sheets[ticker].columns = ['Recent']
    
combined_sheets = pd.concat(recent_sheets)

combined_sheets = combined_sheets.reset_index()
combined_sheets.columns = ['Ticker', 'Breakdown', 'Recent']

In [ ]:
income_statements = {}
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ticker in stock_list:
        income_statements[ticker] = executor.submit(si.get_income_statement,ticker)
#         print(ticker)
print('done')

for ticker in stock_list:
    income_statements[ticker] = income_statements[ticker].result()
#     print(ticker + '2')
    
print('done2')

In [ ]:
recent_income_statements = {ticker : sheet.iloc[:,:1] for ticker,sheet in income_statements.items()}
for ticker in recent_income_statements.keys():
    recent_income_statements[ticker].columns = ['Recent']

combined_income = pd.concat(recent_income_statements)
combined_income = combined_income.reset_index()
combined_income.columns = ['Ticker', 'Breakdown', 'Recent']

In [ ]:
cash_flow_statements = {}
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ticker in stock_list:
        cash_flow_statements[ticker] = executor.submit(si.get_cash_flow,ticker)
#         print(ticker)
print('done')

for ticker in stock_list:
    cash_flow_statements[ticker] = cash_flow_statements[ticker].result()
#     print(ticker + '2')
    
print('done2')

In [ ]:
recent_cash_flow_statements = {ticker : sheet.iloc[:,:1] for ticker,sheet in income_statements.items()}
for ticker in recent_cash_flow_statements.keys():
    recent_cash_flow_statements[ticker].columns = ['Recent']

combined_cash_flow = pd.concat(recent_cash_flow_statements)
combined_cash_flow = combined_cash_flow.reset_index()
combined_cash_flow.columns = ['Ticker', 'Breakdown', 'Recent']

In [ ]:
frames = [combined_sheets, combined_income, combined_cash_flow]
combined_df = pd.concat(frames)
combined_df = combined_df.reset_index()

In [ ]:
pivot_df = combined_df.pivot_table(index='Ticker', columns='Breakdown', values='Recent', aggfunc='sum')

In [ ]:
pivot_df.query("Ticker == 'AMZN'")

In [ ]:
pivot_df.query("ebit >= 0")

In [ ]:
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))